In [121]:
import pandas as pd
import boto3
import pyarrow
import awswrangler as wr

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.live import StockDataStream
from alpaca.data.requests import StockBarsRequest, StockTradesRequest
from alpaca.data.timeframe import TimeFrame

from dotenv import load_dotenv
import os
import json
from datetime import datetime, timedelta

In [122]:
load_dotenv()
ALPACA_API_KEY = os.environ.get("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.environ.get("ALPACA_SECRET_KEY")
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")

## Reading Tickers

In [123]:
nasdaq = pd.read_csv("https://s3-alpaca-stock-data.s3.us-west-1.amazonaws.com/tickers/nasdaq100.csv")
nyse = pd.read_csv("https://s3-alpaca-stock-data.s3.us-west-1.amazonaws.com/tickers/nyse100.csv")

In [124]:
nasdaq.head()

,Ticker,Code,Name
0,FOXA,GOOG/NASDAQ_FOXA,21st Century Fox Class A
1,ATVI,GOOG/NASDAQ_ATVI,Activision Blizzard Inc
2,ADBE,GOOG/NASDAQ_ADBE,Adobe Systems Inc
3,AKAM,GOOG/NASDAQ_AKAM,Akamai Technologies Inc
4,ALXN,GOOG/NASDAQ_ALXN,Alexion Pharmaceuticals Inc


In [125]:
nyse.head()

,Ticker,Code,Name
0,MMM,GOOG/NYSE_MMM,3M Co.
1,ABT,GOOG/NYSE_ABT,Abbott Laboratories
2,ACN,GOOG/NYSE_ACN,Accenture PLC Cl A
3,AGN,GOOG/NYSE_AGN,Allergan Inc.
4,MO,GOOG/NYSE_MO,Altria Group Inc.


In [126]:
nasdaq_tickers = list(nasdaq.iloc[:, 0])
nyse_tickers = list(nyse.iloc[:, 0])
tickers = set(nasdaq_tickers + nyse_tickers)

## Batch Data Ingestion

In [127]:
hist_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

In [129]:
def convert_datetime(date: datetime):
    return datetime(date.year, date.month, date.day, 0, 0, 0)

def getStockHistoricalData(client: StockHistoricalDataClient, start_date, end_date = datetime.today()):
    bar_request = StockBarsRequest(
        symbol_or_symbols=tickers,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date
    )
    res = client.get_stock_bars(bar_request)
    return(res)

bars = getStockHistoricalData(hist_client, datetime(2023, 9, 20)).data
# bars = getStockHistoricalData(hist_client, datetime.today() - timedelta(days = 1)).data

In [137]:
pd.concat([
    nasdaq[nasdaq['Ticker'].isin(list(bars.keys()))], 
    nyse[nyse['Ticker'].isin(list(bars.keys()))]], ignore_index=True).to_csv('all_tickers.csv', index=False)

In [113]:
pd.DataFrame(list(bars.keys()), columns=['Ticker']).to_csv('all_tickers.csv', index=False)

In [117]:
BAR_SCHEMA = {
    "symbol": str,
    "high": "float64",
    "low": "float64",
    "open": "float64",
    "timestamp": "datetime64[ns]",
    "trade_count": "float64",
    "volume": "float64",
    "vwap": "float64"
}
df = pd.DataFrame(columns=BAR_SCHEMA.keys()).astype(BAR_SCHEMA)

for i, ticker in enumerate(bars):
    for bar in bars[ticker]:
        entry = {
            "symbol": bar.symbol,
            "high": bar.high,
            "low": bar.low,
            "open": bar.open,
            "timestamp": bar.timestamp,
            "trade_count": bar.trade_count,
            "volume": bar.volume,
            "vwap": bar.vwap
        }
        df.loc[len(df)] = entry


In [118]:
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day

In [119]:
df.head()

,symbol,high,low,open,timestamp,trade_count,volume,vwap,year,month,day
0,GOOGL,785.0000,765.95,766.94,2015-12-01 05:00:00+00:00,32855.0,2019864.0,779.648791,2015,12,1
1,GOOGL,793.0475,776.43,785.25,2015-12-02 05:00:00+00:00,33353.0,2343841.0,783.248876,2015,12,2
2,GOOGL,784.4800,762.45,780.56,2015-12-03 05:00:00+00:00,33445.0,2097494.0,771.096835,2015,12,3
3,GOOGL,782.4200,765.51,769.71,2015-12-04 05:00:00+00:00,31932.0,2489998.0,777.131097,2015,12,4
4,GOOGL,781.0000,766.72,780.06,2015-12-07 05:00:00+00:00,27740.0,1831287.0,772.486758,2015,12,7


In [120]:
df.tail()

,symbol,high,low,open,timestamp,trade_count,volume,vwap,year,month,day
1959,GOOGL,138.695,136.24,137.60,2023-09-14 04:00:00+00:00,271033.0,24759214.0,137.922042,2023,9,14
1960,GOOGL,138.520,136.48,137.98,2023-09-15 04:00:00+00:00,301744.0,38917834.0,137.423197,2023,9,15
1961,GOOGL,139.160,136.61,136.61,2023-09-18 04:00:00+00:00,265606.0,21865772.0,138.294902,2023,9,18
1962,GOOGL,138.410,136.62,137.42,2023-09-19 04:00:00+00:00,223419.0,20982993.0,137.705468,2023,9,19
1963,GOOGL,138.080,133.62,138.08,2023-09-20 04:00:00+00:00,326713.0,29806954.0,135.335839,2023,9,20


## Using AWS SDK (Main Flow)

In [73]:
glue_db_name = "alpaca_stocks_database"
# glue_table_name = f"stocks_table_{datetime.now().year}_{datetime.now().month}"
glue_table_name = "stocks_table_historical"

session = boto3.Session(
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = "us-west-1"
)

In [77]:
wr.s3.to_parquet(
    df = df,
    path = "s3://s3-alpaca-stock-data/historical/",
    dataset = True,
    partition_cols = ["year", "month", "day"],
    database = glue_db_name,
    table = glue_table_name,
    boto3_session = session,
    mode = "overwrite_partitions"  
)

{'paths': ['s3://s3-alpaca-stock-data/historical/year=2023/month=1/day=3/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=4/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=5/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=6/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=9/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=10/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=11/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=12/f6ff9d9967b04de1adad0b3f81674f5f.snappy.parquet',
  's3://s3-alpaca-stock-data/historical/year=2023/month=1/day=13/f6ff9d9967b

## Transformation

In [31]:
athena_query = f'''
    SELECT *
    FROM {glue_table_name}
    ORDER BY day ASC
    LIMIT 10;
'''

wr.athena.read_sql_query(athena_query, database = glue_db_name, boto3_session = session)

,symbol,high,low,open,timestamp,trade_count,volume,vwap,year,month,day
0,KLAC,508.730,498.430,503.580,2023-09-01 04:00:00,21807.0,657206.0,505.892743,2023,9,1
1,EQIX,791.650,775.825,788.370,2023-09-01 04:00:00,10788.0,234829.0,780.728574,2023,9,1
2,CAT,287.070,283.650,284.800,2023-09-01 04:00:00,46592.0,2336591.0,285.642291,2023,9,1
3,BK,45.755,45.070,45.250,2023-09-01 04:00:00,49345.0,5147646.0,45.463016,2023,9,1
4,AVGO,901.870,864.660,901.870,2023-09-01 04:00:00,205258.0,6501666.0,878.321053,2023,9,1
5,EBAY,45.540,44.870,45.210,2023-09-01 04:00:00,51440.0,4176390.0,45.115157,2023,9,1
6,ADP,256.760,254.050,254.790,2023-09-01 04:00:00,25680.0,1484201.0,255.157668,2023,9,1
7,BAC,29.130,28.760,28.760,2023-09-01 04:00:00,136094.0,35228576.0,28.971747,2023,9,1
8,CL,73.965,73.000,73.950,2023-09-01 04:00:00,30738.0,2412842.0,73.430519,2023,9,1
9,GOOGL,137.455,134.850,137.455,2023-09-01 04:00:00,276157.0,21653686.0,135.693452,2023,9,1


## Using Boto3

In [124]:
df.to_parquet("stocks.parquet.gzip", engine = "pyarrow", compression = "gzip")

In [125]:
s3 = boto3.client("s3")

file_path = "stocks.parquet.gzip"
bucket_name = "s3-alpaca-stock-data"


In [126]:
s3.upload_file(file_path, bucket_name, "historical/test.parquet.gzip")